# Formulação de Carvalho

**Variáveis de decisão:**
* $x_v,_c$ =
  \begin{cases}
  1, \text{se o vértice $v$ recebe a cor $c$}\\
  0, \text{caso contrário}\\
  \end{cases}

* $z_c$ =
  \begin{cases}
  1, \text{se a cor $c$ é usada}\\
  0, \text{caso contrário}\\
  \end{cases}

**Restrições**:

* Restrição de maximização

$$\Gamma_C = \max \sum_{c \in C} z_c$$

* Restrição de garatia de coloração única

$$s.a. \quad \sum_{c \in C} x_{v,c} = 1 \quad \forall v \in V$$

* Restrição de garantia que a coloração obtida será própria

$$x_{v,c} + x_{u,c} \leq z_c \quad \forall \{u,v\} \in E(G), \forall c \in C$$

* Restrição de garantia que uma cor $k$ só pode ser usada se algum vértice $v$ recebeu a cor
$k$.

$$z_c \leq \sum_{v \in V} x_{v,c} \quad \forall c \in C$$

* Restrição que afiança que os vértices isoladoss pode receber qualquer cor.

$$x_{v,c} \leq z_c \quad \forall v \in V(G), \forall c \in \mathcal{C} \\$$

* Restrição que assegura que as primeiras cores serão usadas.

$$z_{c'} \leq z_c \quad \forall c, c' \in \mathcal{C} \text{ se } c < c'$$

* Restrição que garante a propriedade de Grundy $O(|V||C|^2)$ . 

$$x_{v,c'} \leq \sum_{u \in N(v)} x_{u,c} \quad \forall v \in V(G), \forall c, c' \in \mathcal{C} \text{ se } c < c'$$

*

$$x_{v,c} \in \{0,1\} \quad \forall v \in V(G), \forall c \in \mathcal{C}$$

*

$$z_c \in \{0,1\} \quad \forall c \in \mathcal{C}$$




In [1]:
# Instalação de pacotes necessários para execução do código
%pip install ortools
%pip install networkx

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import networkx as nx
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

def coloracao_carvalho(G):  
    # Array de 100 cores fornecido
    color_array = [
        "gold", "red", "violet", "maroon", "limegreen", "greenyellow", "black",
        "blue", "teal", "indigo", "crimson", "turquoise", "coral", "pink",
        "olive", "navy", "magenta", "salmon", "fuchsia", "chartreuse", "skyblue",
        "beige", "sienna", "lavender", "aquamarine", "plum", "steelblue", "khaki",
        "moccasin", "peru", "chocolate", "orchid", "slateblue", "forestgreen",
        "seagreen", "tomato", "turquoise", "mediumvioletred", "cadetblue", "darkcyan",
        "palevioletred", "dodgerblue", "lightseagreen", "lightsalmon", "darkslategray",
        "darkmagenta", "yellowgreen", "mediumpurple", "lightcoral", "springgreen",
        "darkorchid", "powderblue", "rosybrown", "darkgoldenrod", "palegreen", "hotpink",
        "palegoldenrod", "mistyrose", "lightpink", "lime", "midnightblue", "darkseagreen",
        "indianred", "lightgreen", "darkolivegreen", "sandybrown", "mediumslateblue",
        "lightsteelblue", "firebrick", "oceanblue", "lightcyan", "thistle", "lawngreen",
        "royalblue", "seashell", "darkkhaki", "lightskyblue", "navajowhite", "lavenderblush",
        "cornflowerblue", "goldenrod", "darkblue", "darkturquoise", "mediumspringgreen",
        "wheat", "papayawhip", "lightgoldenrodyellow", "peru", "bisque", "peachpuff",
        "lemonchiffon", "lightgray", "mediumorchid", "darkred", "burlywood", "slategray",
        "mediumseagreen", "antiquewhite", "honeydew", "mintcream"
    ]

    # Verificar se o grafo foi corretamente inicializado
    vertices = list(G.nodes())
    n = len(vertices)
    if n == 0:
        raise ValueError("O grafo não foi inicializado corretamente.")

    # Inicializando o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Verificar se o solver foi corretamente inicializado
    if not solver:
        raise ValueError("O solver não foi inicializado corretamente.")

    # Definir o tempo máximo de execução para o solver (30 minutos)
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos em milissegundos

    # Variáveis de decisão
    x = [[solver.IntVar(0.0, 1.0, f'x_{v}_{j}') for j in range(n)] for v in range(n)]  # Se o vértice v recebe a cor j
    z = [solver.IntVar(0.0, 1.0, f'z_{j}') for j in range(n)]  # Se a cor j é usada

    # 1. Maximizar o número de cores usadas
    solver.Maximize(solver.Sum(z[j] for j in range(n)))

    # Restrições
    # (3.2.2) Cada vértice recebe exatamente uma cor
    for v in range(n):
        solver.Add(solver.Sum(x[v][j] for j in range(n)) == 1)

    # (3.2.3) Restrições de coloração própria: vértices adjacentes não podem ter a mesma cor
    for (u, v) in G.edges():
        for j in range(n):
            solver.Add(x[u][j] + x[v][j] <= z[j])

    # (3.2.4) Restrições para garantir que uma cor seja usada somente se algum vértice a usar
    for j in range(n):
        solver.Add(z[j] <= solver.Sum(x[v][j] for v in range(n)))

    # (3.2.5) Os vértices isolados podem receber qualquer cor
    for v in range(n):
        if G.degree(v) == 0:
            for j in range(n):
                solver.Add(x[v][j] == 1)

    # (3.2.6) Assegura que as primeiras cores são usadas
    for j in range(n-1):
        solver.Add(z[j] >= z[j+1])

    # (3.2.7) Restrições para garantir a propriedade Grundy
    for v in range(n):
        for j in range(1, n):
            solver.Add(x[v][j] >= 1 - solver.Sum(x[v][d] for d in range(j)) - solver.Sum(x[u][j] for u in G.neighbors(v)))
        
    # (3.2.8) Esta retrição define que a variável de decisão x v,c pode assumir apenas dois valores: 0 ou 1 (váriáveis de decisão)
    for v in range(n):
        for j in range(n):
            solver.Add(x[v][j] <= 1)
            solver.Add(x[v][j] >= 0)    

    # (3.2.9) Esta retrição define que a variável de decisão z c pode assumir apenas dois valores: 0 ou 1 (váriáveis de decisão)  
    for j in range(n):
        solver.Add(z[j] <= 1)
        solver.Add(z[j] >= 0)

    # Resolver o problema
    print("Resolvendo o problema...")
    status = solver.Solve()

    # Verificar o status da solução
    color_map = []

    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada:')
        print('Valor objetivo =', solver.Objective().Value())

        for v in range(n):
            for j in range(n):
                if x[v][j].solution_value() == 1:
                    color_map.append(color_array[j])
                   

    elif status == pywraplp.Solver.FEASIBLE:
        print('Uma solução viável foi encontrada dentro do limite de tempo.')
        print('Valor objetivo =', solver.Objective().Value())

        for v in range(n):
            for j in range(n):
                if x[v][j].solution_value() == 1:
                    color_map.append(color_array[j])

    else:
        print('O problema não tem uma solução ótima dentro do limite de tempo.')

    print('Problem solved in %f milliseconds' % (solver.WallTime()/1000))
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


In [3]:
import networkx as nx

In [3]:
G = nx.gnp_random_graph(10, 0.5)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 6.0
Problem solved in 3.931000 milliseconds
Problem solved in 2253 iterations
Problem solved in 42 branch-and-bound nodes


In [8]:
G = nx.gnp_random_graph(10, 0.8)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 8.0
Problem solved in 3.320000 milliseconds
Problem solved in 4636 iterations
Problem solved in 49 branch-and-bound nodes


In [10]:
G = nx.gnp_random_graph(13, 0.5)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 8.0
Problem solved in 33.114000 milliseconds
Problem solved in 108265 iterations
Problem solved in 3125 branch-and-bound nodes


In [13]:
G = nx.gnp_random_graph(13, 0.8)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 9.0
Problem solved in 106.434000 milliseconds
Problem solved in 542351 iterations
Problem solved in 15141 branch-and-bound nodes


In [14]:
G = nx.gnp_random_graph(15, 0.5)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 8.0
Problem solved in 39.793000 milliseconds
Problem solved in 166693 iterations
Problem solved in 4079 branch-and-bound nodes


In [15]:
G = nx.gnp_random_graph(15, 0.8)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 11.0
Problem solved in 260.752000 milliseconds
Problem solved in 997367 iterations
Problem solved in 28626 branch-and-bound nodes


In [4]:
G = nx.gnp_random_graph(18, 0.5)
coloracao_carvalho(G)

Resolvendo o problema...
Solução ótima encontrada:
Valor objetivo = 10.000000000000007
Problem solved in 493.033000 milliseconds
Problem solved in 2269501 iterations
Problem solved in 23081 branch-and-bound nodes


In [7]:
G = nx.gnp_random_graph(18, 0.8)
coloracao_carvalho(G)

Resolvendo o problema...
Uma solução viável foi encontrada dentro do limite de tempo.
Valor objetivo = 13.0
Problem solved in 1799.307000 milliseconds
Problem solved in 7176671 iterations
Problem solved in 79763 branch-and-bound nodes


In [8]:
G = nx.gnp_random_graph(20, 0.5)
coloracao_carvalho(G)

Resolvendo o problema...
Uma solução viável foi encontrada dentro do limite de tempo.
Valor objetivo = 11.0
Problem solved in 1799.252000 milliseconds
Problem solved in 6480299 iterations
Problem solved in 76464 branch-and-bound nodes


In [9]:
G = nx.gnp_random_graph(20, 0.8)
coloracao_carvalho(G)   

Resolvendo o problema...
Uma solução viável foi encontrada dentro do limite de tempo.
Valor objetivo = 13.0
Problem solved in 1800.002000 milliseconds
Problem solved in 6234761 iterations
Problem solved in 34643 branch-and-bound nodes
